In [1]:
import csv
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
raw = pd.read_csv("1429_1.csv")

C:\Users\Vinot\AppData\Local\Temp\ipykernel_2812\1528679751.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv("1429_1.csv")


In [3]:
raw.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [4]:
textdata = raw[['categories','name','reviews.text','reviews.rating','reviews.title']]
textdata.shape

(34660, 5)

In [5]:
textdata= textdata.dropna()

In [6]:
textdata

,categories,name,reviews.text,reviews.rating,reviews.title
0,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",This product so far has not disappointed. My c...,5.0,Kindle
1,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",great for beginner or experienced person. Boug...,5.0,very fast
2,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Inexpensive tablet for him to use and learn on...,5.0,Beginner tablet for our 9 year old son.
3,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I've had my Fire HD 8 two weeks now and I love...,4.0,Good!!!
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I bought this for my grand daughter when she c...,5.0,Fantastic Tablet for kids
...,...,...,...,...,...
27895,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",New Amazon Kindle Fire Hd 9w Powerfast Adapter...,This is my new favorite device. While not perf...,5.0,My new favorite product
27896,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",New Amazon Kindle Fire Hd 9w Powerfast Adapter...,I got this to basically experiment with. Strai...,5.0,Lots of potential!!!
27897,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",New Amazon Kindle Fire Hd 9w Powerfast Adapter...,Good product that does the basics. Too bad you...,4.0,Good First Generation Product
27898,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",New Amazon Kindle Fire Hd 9w Powerfast Adapter...,This is great for a connected home. People who...,3.0,"Great for a ""connected home"""


In [7]:
stop=stopwords.words('english')

In [8]:
#removing stop words

textdata['review_body']=textdata["reviews.text"].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [9]:
textdata.head()

,categories,name,reviews.text,reviews.rating,reviews.title,review_body
0,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",This product so far has not disappointed. My c...,5.0,Kindle,This product far disappointed. My children lov...
1,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",great for beginner or experienced person. Boug...,5.0,very fast,great beginner experienced person. Bought gift...
2,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Inexpensive tablet for him to use and learn on...,5.0,Beginner tablet for our 9 year old son.,"Inexpensive tablet use learn on, step NABI. He..."
3,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I've had my Fire HD 8 two weeks now and I love...,4.0,Good!!!,I've Fire HD 8 two weeks I love it. This table...
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I bought this for my grand daughter when she c...,5.0,Fantastic Tablet for kids,I bought grand daughter comes visit. I set use...


In [10]:
textdata['star_rating']=textdata['reviews.rating'].astype(float).astype(int)

In [11]:
textdata.head()

,categories,name,reviews.text,reviews.rating,reviews.title,review_body,star_rating
0,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",This product so far has not disappointed. My c...,5.0,Kindle,This product far disappointed. My children lov...,5
1,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",great for beginner or experienced person. Boug...,5.0,very fast,great beginner experienced person. Bought gift...,5
2,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Inexpensive tablet for him to use and learn on...,5.0,Beginner tablet for our 9 year old son.,"Inexpensive tablet use learn on, step NABI. He...",5
3,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I've had my Fire HD 8 two weeks now and I love...,4.0,Good!!!,I've Fire HD 8 two weeks I love it. This table...,4
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I bought this for my grand daughter when she c...,5.0,Fantastic Tablet for kids,I bought grand daughter comes visit. I set use...,5


In [12]:
textdata['review_body']= textdata['review_body'].str.replace('<br />', r'')

In [13]:
textdata['review_body'] = textdata['review_body'].str.replace("[,'!;.–`:?><-]",'')

C:\Users\Vinot\AppData\Local\Temp\ipykernel_2812\1051208827.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  textdata['review_body'] = textdata['review_body'].str.replace("[,'!;.–`:?><-]",'')


In [14]:
textdata['review_body'] = textdata['review_body'].str.lower()

In [15]:
textdata['review_body'] =textdata.review_body.str.replace('[^a-zA-Z ]', '') 

C:\Users\Vinot\AppData\Local\Temp\ipykernel_2812\4101880381.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  textdata['review_body'] =textdata.review_body.str.replace('[^a-zA-Z ]', '')


In [16]:
textdata.isna().sum()

categories        0
name              0
reviews.text      0
reviews.rating    0
reviews.title     0
review_body       0
star_rating       0
dtype: int64

In [17]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vinot\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:

import nltk
nltk.download('wordnet')
def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    for i in range(0,len(corpus)):
       corpus.iloc[i] =' '.join([lemmatizer.lemmatize(word) for word in corpus.iloc[i].split()])
get_lemmatized_text(textdata['review_body'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vinot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Vinot\AppData\Local\Temp\ipykernel_2812\895541218.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus.iloc[i] =' '.join([lemmatizer.lemmatize(word) for word in corpus.iloc[i].split()])


In [19]:
def partition(x):
    if x<3:
        return 'Negative'
    elif x>3:
        return 'Positive'
    return 'Neutral'

actualScore = textdata['star_rating']
positiveNegative = actualScore.map(partition)
textdata['star_rating'] = positiveNegative

In [20]:
textdata.head()

,categories,name,reviews.text,reviews.rating,reviews.title,review_body,star_rating
0,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",This product so far has not disappointed. My c...,5.0,Kindle,this product far disappointed my child love us...,Positive
1,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",great for beginner or experienced person. Boug...,5.0,very fast,great beginner experienced person bought gift ...,Positive
2,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",Inexpensive tablet for him to use and learn on...,5.0,Beginner tablet for our 9 year old son.,inexpensive tablet use learn on step nabi he t...,Positive
3,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I've had my Fire HD 8 two weeks now and I love...,4.0,Good!!!,ive fire hd two week i love it this tablet gre...,Positive
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",I bought this for my grand daughter when she c...,5.0,Fantastic Tablet for kids,i bought grand daughter come visit i set user ...,Positive


In [21]:
x_train, x_test, y_train, y_test = train_test_split(textdata.review_body, textdata.star_rating, random_state=0,test_size=0.2)

In [22]:

clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression()),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy on train 0.969719
Accuracy on test 0.932173


In [23]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9151988894865634
recall: 0.9321729768526825
fscore: 0.9220308847571528
support: None


In [43]:
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=500,C=0.1,random_state=40,solver='newton-cg')),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.928536
Accuracy on test 0.935762


In [44]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.8756499746010161
recall: 0.9357617082361386
fscore: 0.9047084368653064
support: None


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', RandomForestClassifier()),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999910
Accuracy on test 0.936479


In [27]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9225888720099795
recall: 0.9364794545128298
fscore: 0.9072171036618063
support: None


In [45]:
from sklearn.naive_bayes import MultinomialNB
clf = Pipeline([
    ('vect',TfidfVectorizer()),
    ('clf',MultinomialNB(alpha = 0.1)),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.941501
Accuracy on test 0.936300


In [46]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9089556777691793
recall: 0.9363000179436569
fscore: 0.9074296116403302
support: None


In [30]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier()

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [31]:
import xgboost as xgb
clf = Pipeline([
    ('vect',TfidfVectorizer()),
    ('clf',xgb_classifier),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:40:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy on train 0.961419
Accuracy on test 0.938633


C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [32]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9117584090335844
recall: 0.9386326933429033
fscore: 0.9184418241506677
support: None


C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [33]:

clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression()),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.938047
Accuracy on test 0.937377


In [34]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9089595186540759
recall: 0.9373766373586937
fscore: 0.9150460187235453
support: None


In [35]:
clf = Pipeline([
    ('vect', TfidfVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=500,C=0.1,random_state=40,solver='newton-cg')),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.928402
Accuracy on test 0.935762


In [36]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.8756499746010161
recall: 0.9357617082361386
fscore: 0.9047084368653064
support: None


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
from sklearn.ensemble import RandomForestClassifier
clf = Pipeline([
    ('vect', TfidfVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', RandomForestClassifier()),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999910
Accuracy on test 0.936121


In [38]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9190638593093549
recall: 0.9361205813744842
fscore: 0.9060161953453989
support: None


In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = Pipeline([
    ('vect',CountVectorizer()),
    ('clf',MultinomialNB()),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.939213
Accuracy on test 0.937377


In [40]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9148878891541169
recall: 0.9373766373586937
fscore: 0.9147208279325212
support: None


In [41]:
import xgboost as xgb
clf = Pipeline([
    ('vect',CountVectorizer()),
    ('clf',xgb_classifier),
])
clf = clf.fit(x_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(x_train))
accuracy_test=accuracy_score(y_test, clf.predict(x_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:42:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy on train 0.957965
Accuracy on test 0.938094


C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [42]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = clf.predict(x_test)
y_test 

precision, recall, fscore, support = score(y_test, predicted,average='weighted')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: 0.9121173691347093
recall: 0.9380943836353849
fscore: 0.9187468304735299
support: None


C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
